# Tensorflowで大規模なデータを用いた<br>多クラス分類問題を解くための工夫 <br><br> @Intel0tw5727

## TL;DR
---
+ 約3000クラスの画像を分類するモデルを~~構築しました~~OCに向けて構築中
+ データが大きすぎる問題を様々な方面から解決するために
    + 画像処理アプローチ
        + -> サイズをできる限り落としていこう
    + メモリ管理アプローチ 
        + -> 不要なデータはメモリから削除していこう
    + モジュールによるアプローチ 
        + -> fit_generator をつかっていこう

## 謝罪
---
+ 解説はわかりやすくしていますが、もし間違っていたらぜひマサカリ片手にコメント待ってます
+ 今回のイベントは「Tensorflow-usergroup Okinawa」ですが、これから先、紹介するコードはほぼ *Keras* が中心になっております
+ backend が「 *Tensorflow* 」になっていますのでお許しを

In [2]:
# 証明
from keras import backend as K

# 証明完了 -> Q.E.D
K._BACKEND

'tensorflow'

## 自己紹介

+ 多和田真悟(@Intel0tw5727)
    + 琉球大学 工学部 情報工学科 4年次 遠藤研所属
    + PythonBeginners沖縄 共同オーガナイザー
    + PyData.Okinawa 共同オーガナイザー <- New!(2018/04)
    + ちゅらデータ株式会社 長期インターン生
        + 来月で設立一周年になります🎉
+ 最近の悩み
    + 白米🍚が美味しすぎて腹334分目まで食べてしまうこと

## 今回扱った分類問題

+ ETL手書き文字データベースの第一水準漢字 + ひらがな
    + 3036文字（JIS第１水準漢字：2965，ひらがな：71）
<img src="./fig/etl.png" width=1000>

## 今回扱った分類問題
+ 3036文字の画像を分類する多クラス問題
    + データが大きすぎる
        + 64×63のRGB画像 × 3026文字 × 約20人分
    + 各ラベルごとのデータが足りない
        + 1ラベル20枚前後では学習に苦しさがある
        
    <img src="./fig/etl_example.png">

## 要約

# データを増やしたい！<br>メモリにのせたい！

# 画像処理によるアプローチ

## 画像処理によるアプローチ
+ 今回の問題は白黒画像
    + なのに元データはRGB画像なのでまずはGRAY画像に変換
    + 64\*63 の画像データを28\*28まで削減
    + PythonならOpenCVでできる(PILはﾁｮｯﾄﾜｶﾝﾅｲ)

In [ ]:
# 例
img_prop_list = []
for img in img_list:
    img_prop = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # グレースケール化
    blur = cv2.GaussianBlur(gray_img,(5,5),0) # ガウシアンフィルタで平滑化
    _, img_prop = cv2.threshold(img_prop,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) # 大津法で2値化
    img_prop_list.append(cv2.resize(img_prop, (width, height)).astype("float32") / 255.0) # サイズを削減して正規化

# メモリ管理アプローチ

## メモリ管理アプローチ
+ 普段は気にしていない変数に格納したデータも大規模となると注意が必要
+ 使わない変数はバシバシ消していこう
    + 「del 変数」でメモリを開放できるのでおすすめ
+ 変数のサイズはsysの中のgetsizeof関数で確認できます

In [3]:
import sys
a = [_ for i in range(10000000)]

In [4]:
sys.getsizeof(a)

81528056

# モジュールによるアプローチ

## モジュールによるアプローチ

+ 学習するタイミングですべての訓練データを読み込ませたくない
    + fit_generatorを使うことで訓練データをミニバッチにしてepochごとに動的にデータを読み込むことでメモリを圧迫させない
    + (Tensorflowも、tensorflow.keras.model下にfit_generatorがある)

In [ ]:
# 例
model.fit_generator(datagen.flow(X_train, Y_train,
                                batch_size=256),
                                steps_per_epoch=len(X_train) / 241,
                                epochs=1000,
                                verbose=1,
                                validation_data = (X_test, Y_test),
                                workers=4)

## datagen.flowってなんだこいつ？

In [ ]:
# 例
model.fit_generator(datagen.flow(X_train, Y_train,
                                batch_size=256),
                                steps_per_epoch=len(X_train) / 241,
                                epochs=1000,
                                verbose=1,
                                validation_data = (X_test, Y_test),
                                workers=4)

## datagen.flowとは ImageDataGenerator のことだったのだ
+ kerasモジュールの中にあるこのクラスは訓練やテストに使うデータのかさ増し(Augmentation)を簡単に行い、データをそのままfit_generatorに投げられる優れもの

In [ ]:
# 例
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=5, # 画像の回転幅
        width_shift_range=0.1, # 画像のシフトできる割合
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.1,
        fill_mode='nearest',
        rescale=None,
        preprocessing_function=None,
        data_format="channels_last",
        validation_split=0.2)

##  fit_generator + ImageDataGeneratorで改善

<img src="./fig/bf.png" width=1200>

## 3つのアプローチで改善
+ 改善前
<img src="./fig/before.png">

+ 改善後
<img src="./fig/after.png">

# まとめ
+ 画像の無駄な部分や使わない変数はガンガン削ぎ落としましょう
+ fit_generator +αのアプローチだけでも大規模な学習を回すことができます
    + ~~ただし学習が早く終わるとは言ってない~~
+ ImageDataGeneratorに頼ればラベルごとのデータが少なくてもカバーできます
    + ~~ただし学習が早く終わると(ry~~
+ Tensorflow~~(Keras)~~は便利で素晴らしい！
    + ~~ただし学習(ry~~

# ご静聴ありがとうございました！